In [ ]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("/kaggle/input/default-of-credit-card-clients-dataset/UCI_Credit_Card.csv", index_col="ID")

In [ ]:
df

# Kontrol

## Anormallikler

İçe aktarılan veri seti içerisindeki gözlemlerde değişkenlerin aldığı değerlerin veri sağlayıcısı tarafından verilen bilgiler ile uyuşup uyuşmadığının kontrolü.

In [ ]:
# Kategorik veriler için kurallar
categoric_attribute_information = {
    "SEX": [1,2],
    "EDUCATION": [1, 2, 3, 4],
    "MARRIAGE": [1, 2, 3]
}

for column in categoric_attribute_information.keys():
    unique_values = list(set(df[column].tolist()))
    if unique_values == categoric_attribute_information[column]:
        print("DOĞRU", column, categoric_attribute_information[column], "==", unique_values, "\n")
    else:
        print("HATA!", column, categoric_attribute_information[column], "!=", unique_values, "\n")

Burada cinsiyet verisini içeren değişkene ait gözlemlerde bir anormallik olmadığı tespit edildi fakat hem eğitim seviyesi hem de medeni durum değişkenlerine ait gözlemlerde ortak olarak 0 kategorisi veri sağlayıcısı tarafından verilen kategoriler arasında yer almamaktadır. Bu kategori eğitim seviyesi ve medeni durumun belirtilmediğini ifade ediyor olabilir.

Eğitim durumunda ise diğer diye bir kategori olmasına rağmen 5 ve 6 olmak üzere fazladan iki kategori daha olduğu tespit edildi. Daha sonradan eklenmiş kategoriler olabilir. Bunlar ile tüm anormallikleri tespit ettikten sonra ilgileneceğim.
<hr>
Veri sağlayısı tarafından yaş değişkeninin alabileceği değerler hakkında bir bilgi verilmemiştir. Fakat bu veri setinin Taiwanda kredi kartı sahibi insan olduğunu bilerek basitçe mümkün olabilecek minimum ve maksimum yaşı araştırabiliriz. Aşağıda linki verilmiş kaynakta Taiwanda kredi kartı alabilmek için gerekli minimum yaş kriterinin 20 olduğu belirtilmekte.

https://www.hsbc.com.tw/en-tw/credit-cards/faq/

In [ ]:
print("En küçük yaş:", min(df["AGE"].tolist()), "\n")
print("En büyük yaş:", max(df["AGE"].tolist()), "\n")

Gözlemler arasındaki en küçük yaş kurala uyduğundan burada bir problem yok. En büyük yaş için bir kural yok fakat 79 değeri makul görünüyor.
<hr>
Veri sağlayısı tarafından PAY_0 - ... - PAY_6  değişkenleri için verilen kural aşağıdaki gibidir:

$$ 
test(x) = \left\{
\begin{array}{}
      True & x=-1 \\
      True & x>0\\
\end{array} 
\right.
$$

In [ ]:
columns_starts_with_pay = [column for column in df.columns if column[:4] == "PAY_" and len(column) == 5]

for column in columns_starts_with_pay:
    unique_values = list(set(df[column].tolist()))
    for value in unique_values:
        if not ((value == -1) or (value > 0)):
            print("HATA!", column, "=", value)

Veri sağlayısı tarafından bu değişkenler için verilen alınabilecek değerler arasında 0 ve -2 verilmemiştir. Bu değerlerin de ne anlama geldiğini bilmiyoruz. Aşağıdaki linkte bu veri seti hakkında veri sağlayısından alınan ekstra bilgiler mevcuttur.

https://www.kaggle.com/uciml/default-of-credit-card-clients-dataset/discussion/34608#latest-325928

Bu kaynağa göre anormallik tespit ettiğimiz değişkenler için veri sağlayıcısının verdiği bilgiler aşağıdaki gibidir;

- X3: Education (1 = graduate school; 2 = university; 3 = high school; 0, 4, 5, 6 = others).
- X4: Marital status (1 = married; 2 = single; 3 = divorce; 0=others).
- X6 - X11: History of past payment (-2: No consumption; -1: Paid in full; 0: The use of revolving credit; 1 = payment delay for one month; 2 = payment delay for two months; . . .; 8 = payment delay for eight months; 9 = payment delay for nine months and above).
Burada eğitim seviyesi değişkeninde 0, 4, 5, 6 kategorilerini tek kategori altında toplamak haricinde yapılacak birşey yok gibi görünüyor.

Ayrıca bu veri seti içerisindeki gözlemlerin hem bireysel kredi kartı sahibi müşterilere hem de ek karta sahip müşterilere ait olabileceği bilgisi de verilmiştir bu durumda Taiwan da kredi kartı alma koşulları arasında minimum yaşın ek kart için 16 olduğu bilgisine dayanarak veri seti içerisindeki yaş aralığını tekrardan doğrulamış olduk.

In [ ]:
def combine_categories(x):
    if x in [4, 5, 6]:
        return 0
    else:
        return x

df["EDUCATION"] = df["EDUCATION"].apply(
    lambda x: combine_categories(x)
)

In [ ]:
list(set(df["EDUCATION"].tolist()))

## Eksik Veri
Veri seti içerisinde eksik değer içeren bir değişken ve gözlem yoktur.

In [ ]:
df.info()

# Ön işleme
## Sayısallaştırma

Bu veri seti içerisindeki tüm değişkenler sayısaldır yani bir sayısallaştırma gereksinimi yoktur. Fakat aslında kategorik olan medeni hal değişkeni içerisindeki verilerin birbiri üzerinde bir büyüklük durumu olmadığından bu değişkeni ve ayrıca eğitim seviyesi değişkenini sayısaldan kategoriğe dönüştürüp one hot encoder ile tekrar sayısallaştıracağım.

In [ ]:
# Eğitim seviyesi kategorileri
education_categories = {
    0: "others",
    1: "graduate_student",
    2: "university",
    3: "high_school",
}
# Medeni durum kategorileri
# Bu kategoriler yukarıda linkini verdiğim kaynağa göre veri sağlayıcısı tarafından sonradan düzeltilen kategorilerdir
marriage_categories = {
    0: "others",
    1: "married",
    2: "single",
    3: "divorce"
}

for category in education_categories.keys():
    df["EDUCATION"][df["EDUCATION"] == category] = education_categories[category]
    
for category in marriage_categories:
    df["MARRIAGE"][df["MARRIAGE"] == category] = marriage_categories[category]
    
df

In [ ]:
# One Hot Encoder
categoric_variables = df.select_dtypes(
    include=[np.object]
).columns.tolist()
df = pd.get_dummies(df, prefix=categoric_variables)

# Hedef değişkeni sayısaldan kategorik tipe dönüştürüldü
df["default.payment.next.month"][df["default.payment.next.month"] == 1] = "Yes"
df["default.payment.next.month"][df["default.payment.next.month"] == 0] = "No"

df

## Normalizasyon

In [ ]:
# 0 - 1 Normalizasyon
numeric_variables = df.select_dtypes(
    exclude=[np.object]
).columns.tolist()
for variable in numeric_variables:
    data = list(set(df[variable].tolist()))
    min_value = min(data)
    max_value = max(data)
    df[variable] = df[variable].apply(
        lambda x: (x-min_value)/(max_value-min_value)
    )
    
df

# Modelleme
## Hedef Kategorilerin Dağılımı
Hedef değişkenin kategorileri arasındaki orana bakıldığında dengesiz bir dağılım olduğu görülmektedir. Bu durum oluşacak modelin başarısını negatif etkileyecektir.

In [ ]:
count_yes = df["default.payment.next.month"].tolist().count("Yes")
count_no = df["default.payment.next.month"].tolist().count("No")
print("Yes sayısı:", count_yes)
print(" No sayısı:", count_no)
print(" Yes oranı:", count_yes / (count_yes+count_no))

## Verinin Eğitim İçin Hazırlanması

In [ ]:
df = df.rename(columns={"default.payment.next.month": "target"})

X = df.drop(columns=["target"]).values
y = df.filter(["target"])
y = pd.get_dummies(y, prefix=["target"]).values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42
)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
y_train.shape

In [ ]:
y_test.shape

In [ ]:
split = int(len(y_test)/2)
inputX = X_train
inputY = y_train
inputX_valid = X_test[:split]
inputY_valid = y_test[:split]
inputX_test = X_test[split:]
inputY_test = y_test[split:]

## Model

In [ ]:
input_nodes = inputX.shape[1]

multiplier = 3

hidden_nodes1 = input_nodes
hidden_nodes2 = round(hidden_nodes1 * multiplier)
hidden_nodes3 = round(hidden_nodes2 * multiplier)

pkeep = tf.placeholder(tf.float32)

# Input
x = tf.placeholder(tf.float32, [None, input_nodes])

# Layer 1
W1 = tf.Variable(tf.truncated_normal([input_nodes, hidden_nodes1], stddev = 0.15))
b1 = tf.Variable(tf.zeros([hidden_nodes1]))
y1 = tf.nn.sigmoid(tf.matmul(x, W1) + b1)

# Layer 2
W2 = tf.Variable(tf.truncated_normal([hidden_nodes1, hidden_nodes2], stddev = 0.15))
b2 = tf.Variable(tf.zeros([hidden_nodes2]))
y2 = tf.nn.sigmoid(tf.matmul(y1, W2) + b2)

# Layer 3
W3 = tf.Variable(tf.truncated_normal([hidden_nodes2, hidden_nodes3], stddev = 0.15)) 
b3 = tf.Variable(tf.zeros([hidden_nodes3]))
y3 = tf.nn.sigmoid(tf.matmul(y2, W3) + b3)
y3 = tf.nn.dropout(y3, pkeep)

# Layer 4
W4 = tf.Variable(tf.truncated_normal([hidden_nodes3, 2], stddev = 0.15)) 
b4 = tf.Variable(tf.zeros([2]))
y4 = tf.nn.softmax(tf.matmul(y3, W4) + b4)

# Output
y = y4
y_ = tf.placeholder(tf.float32, [None, 2])

training_epochs = 100
training_dropout = 0.9
display_step = 10
n_samples = y_train.shape[0]
batch_size = 2048
learning_rate = 0.01

# Cross entropy
cost = -tf.reduce_sum(y_ * tf.log(y))

optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

accuracy_summary = []
cost_summary = []
valid_accuracy_summary = [] 
valid_cost_summary = [] 
stop_early = 0

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(training_epochs):
        for batch in range(int(n_samples/batch_size)):
            batch_x = inputX[batch*batch_size : (1+batch)*batch_size]
            batch_y = inputY[batch*batch_size : (1+batch)*batch_size]
            
            sess.run([optimizer], feed_dict={
                x: batch_x,
                y_: batch_y,
                pkeep: training_dropout
            })
            
        if (epoch) % display_step == 0:
            train_accuracy, newCost = sess.run([accuracy, cost], feed_dict={
                x: inputX,
                y_: inputY,
                pkeep: training_dropout
            })
            
            valid_accuracy, valid_newCost = sess.run([accuracy, cost], feed_dict={
                x: inputX_valid,
                y_: inputY_valid,
                pkeep: 1
            })
            
            print(
                "Epoch:", epoch,
                "Acc =", "{:.5f}".format(train_accuracy),
                "Cost =", "{:.5f}".format(newCost),
                "Valid_Acc =", "{:.5f}".format(valid_accuracy),
                "Valid_Cost = ", "{:.5f}".format(valid_newCost)
            )
            
            accuracy_summary.append(train_accuracy)
            cost_summary.append(newCost)
            valid_accuracy_summary.append(valid_accuracy)
            valid_cost_summary.append(valid_newCost)
            
            if valid_accuracy < max(valid_accuracy_summary) and epoch > 100:
                stop_early += 1
                if stop_early == 15:
                    break
            else:
                stop_early = 0
                
    print("Optimization Finished!")

## Model Başarısı
Test için olan cost ve accuracy mavi renkte, validasyon için gösterilen cost ve accuracy ise yeşil renktedir.

In [ ]:
fig, axes = plt.subplots(2, sharex=True, figsize=(12, 8))

axes[0].set_ylabel("Cost", fontsize=14)
axes[0].plot(cost_summary, color='blue')
axes[0].plot(valid_cost_summary, color='green')

axes[1].set_ylabel("Accuracy", fontsize=14)
axes[1].set_xlabel("Epoch", fontsize=14)
axes[1].plot(accuracy_summary, color='blue')
axes[1].plot(valid_accuracy_summary, color='green')
plt.show()

Tensorflow model inspired by [Ilias Siamplis](https://www.kaggle.com/isiablis)